# 1. Overview
In this assignment, you will perform a first-principles electronic structure calculation for the semiconductor GaAs using Quantum ESPRESSO.
You will:

- Run a self-consistent field (SCF) calculation
- Run a non-self-consistent (NSCF) band calculation
- Perform post-processing with bands.x
- Generate a band structure diagram with python
- Understand the input files, pseudopotentials, and k-point sampling

The exercise strengthens your understanding of plane-wave DFT workflows and band-structure analysis.

# 2. Files Provided
You can download the following files from LEARN Lab003_Assigment.

## Input Files
- GaAs.scf.in
- GaAs.nscf.in ( to be edited with the number of band)
- GaAs.band.in

## Pseudopotentials
- Ga.pbe-dn-kjpaw_psl.1.0.0.UPF
- As.pbe-n-kjpaw_psl.1.0.0.UPF

## plots
- plot.py ( to be edited with the fermi energy)

# 3. Task Instructions
Download and extract the prepared script directory:

# 4. Understanding the Input Files
## 4.1 SCF Input (GaAs.scf.in)
- &control — calculation type, prefix, file paths
- &system — lattice, atomic types, cutoff energies
- &electrons — convergence settings
- ATOMIC_SPECIES — elements + pseudopotentials
- ATOMIC_POSITIONS — internal coordinates
- K_POINTS {automatic} — 8×8×8 grid

## Key physics concepts:
- GaAs uses zinc-blende structure (fcc-based).
- Cutoff energies (ecutwfc=60, ecutrho=244) follow pseudopotential  recommendations.
- SCF uses a coarse k-point grid because only electron density is required.

## 4.2 NSCF Input (GaAs.nscf.in)
Changes from SCF:

- calculation = 'bands'
- Added nbnd = xxx
- K_POINTS {crystal_b} — piecewise-defined high-symmetry path:
- L → Γ → X → K/U → Γ

Purpose: Use the SCF density to compute eigenvalues along the band path.

### Changes Needed for a Band-Structure (NSCF) Calculation
To compute a band structure, we start from the converged SCF density and run a non-self-consistent (NSCF) calculation along a high-symmetry k-path. This requires several key changes to the SCF input.

`calculation = 'bands'`
Tells the code to:

- Use the SCF charge density
- Compute eigenvalues only
- Follow a manually defined k-path instead of a uniform k-grid

`nbnd = xxx`
Specifies the number of electronic bands to compute.

- SCF only requires enough occupied bands
- Band plots need extra conduction bands
- Choosing ~2–3× the number of occupied bands is typical

### 2. How to Determine nbnd
Quantum ESPRESSO automatically determines the number of occupied bands from the valence electrons.

Number of electrons 𝑁𝑒=sum of pseudopotential valence electrons

Then the number of occupied bands is: 𝑁occ=⌈𝑁𝑒/2⌉
 (for spin-unpolarized systems)

Example

If your system has 20 valence electrons per primitive cell: 𝑁occ=202=10occupied bands

For a band structure calculation, you also need extra empty (conduction) bands.

Rule of Thumb

Insulators:
𝑛𝑏𝑛𝑑=𝑁occ+4–8

Metals:
𝑛𝑏𝑛𝑑=𝑁occ+8–16

Complex systems:
𝑛𝑏𝑛𝑑=2𝑁occ

How to Check Automatically

After the SCF run completes, QE prints:

```
number of Kohn-Sham states = XX
```
This value is the minimum nbnd. Add the recommended number of conduction bands on top.

K_POINTS {crystal_b}
This block defines the high-symmetry k-path used in band-structure plots.

Example path for GaAs: L → Γ → X → K/U → Γ


# 5 . Post-Processing
GaAs.band.in (for bands.x) contains:

```
&amp;bands
prefix = 'GaAs'
lsym = .true.
/
```

This generates bands.out.gnu, which is plotted by plot.gp or by plot.py

# 6. Task Instructions Running the Calculations (HPC / SLURM )
This section explains how to run the SCF, NSCF, and band‐structure post-processing steps for GaAs using Quantum ESPRESSO on an HPC cluster with SLURM. All commands below assume you are inside the directory containing:

- GaAs.scf.in ( to be edited )
- GaAs.nscf.in ( to be edited with the number of band)
- GaAs.band.in
- plot.py ( to be edited with the fermi energy)
- Pseudopotentials (Ga.UPF, As.UPF)
- Any job scripts you prepare (run_scf.slurm, run_nscf.slurm, etc.)

## Step 1 — SCF Calculation
modify the GaAs.scf.in : add `verbosity = 'high' in the &control and

Submit the self-consistent calculation to SLURM:

```
sbatch run_scf.slurm
```

Where your run_scf.slurm job script should contain something like:
```
#!/bin/bash
#SBATCH --job-name=GaAs_scf
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --time=00:10:00

module load quantum-espresso

mpirun pw.x < GaAs.scf.in > GaAs.scf.out
```

## Step 2 — NSCF Band Calculation
After the SCF job finishes and produces a converged charge density (GaAs.save/),

Look in GaAs.scf.out

Search for:
```
the Fermi energy is   
```

or

```
highest occupied level (ev): 
```

Also search for

```
number of Kohn-Sham states = XX
```

then edit

`GaAs.nscf.in`

```
nbnd = XX + Nextra
```

submit:

```
sbatch run_nscf.slurm
```

Example NSCF job script:
```
#!/bin/bash
#SBATCH --job-name=GaAs_nscf
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --time=00:10:00

module load quantum-espresso

mpirun pw.x < GaAs.nscf.in > GaAs.nscf.out
```

## Step 3 — Band Post-Processing
Once NSCF completes:
```
sbatch run_bands.slurm
```
Example:
```
#!/bin/bash
#SBATCH --job-name=GaAs_bands
#SBATCH --nodes=1
#SBATCH --ntasks=4
#SBATCH --time=00:05:00

module load quantum-espresso

mpirun bands.x < GaAs.band.in > GaAs.band.out
```

This generates the file GaAs.band.gnu for plotting.

## Step 4 — Plotting the Band Structure
python   plot.gp
This will produce a band-structure figure (e.g., band.png or band.eps depending on your plot.gp settings).

# 7. Assignment Questions
Answer the following in a short report :

1. What Is the fermi energy?
2. Why is the SCF calculation performed on a coarse k-point grid, while the NSCF calculation uses a dense path?
3. Is GaAs a direct or indirect semiconductor based on your calculation?
4. How did you decide on the number of bands?

# 8. Deliverables
Upload the following:

Your band structure plot (PNG )
Short Answer to the questions